### 로그인 & 회원가입 구현

#### 1. account 앱 생성
- python manage.py startapp accounts

#### 2. settings.py에 추가
- 앱을 생성할 경우 -> 반드시 settings.py에 알리기

In [ ]:
# tutorial -> settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'blog',
    'accounts', # 추가
]

#### 3. 모델 형성하기


- 유저 모델을 형성
- 로그인, 회원가입을 위한 모델 형성
- 모델 형성후 반드시 migration 해주기
    - 기존에 superuser의 데이터 때문에 오류 발생 가능
    - admin과 관련된 부분 주석처리 후 migration

In [ ]:
# accounts -> models.py

from django.db import models
from django.contrib.auth.models import AbstractUser

# Create your models here.
class CustomUser(AbstractUser):
    nickname = models.CharField(max_length=100)
    university = models.CharField(max_length=50)
    location = models.CharField(max_length=200)

#### 4. settings.py에 알리기
- accounts 앱에서 만든 CustomUser 모델을 사용하기 위해 장고에 알려야함

In [ ]:
# tutorial -> settings.py

AUTH_USER_MODEL = 'accounts.CustomUser' # 추가해주기

#### 5. admin 페이지에 모델 등록하기
- admin 페이지에서 데이터를 관리할 수 있음
- 가입된 회원 정보를 관리하기 위해 accounts앱의 admin 페이지에 CustomUser 모델을 등록

In [ ]:
# accounts -> admin.py

from django.contrib import admin
from .models import CustomUser
# Register your models here.

admin.site.register(CustomUser) # CustomUser 모델을 등록

#### 6. serializers 정의하기
- accounts앱에 serializers.py 생성
- blog의 serializer처럼 accounts의 serializers를 생성

In [ ]:
# accounts -> serializers.py 

from rest_framework import serializers
from .models import CustomUser # 모델 사용

class LoginSerializer(serializers.Serializer): # 로그인
    username = serializers.CharField(max_length=150)
    password = serializers.CharField(max_length=128)

class SignupSerializer(serializers.ModelSerializer): # 회원가입
    class Meta:
        model = CustomUser # 모델에 대한 정의가 필요 -> ModelSerializer을 사용하기 때문
        fields = ['username', 'password', 'nickname', 'university', 'location', ]

#### 7. blog앱에 author 추가하기
- BlogSerializer 에 author 추가
- 블로그 model에 author 필드 추가

In [ ]:
# blog -> serializers.py

class BlogSerializer(serializers.ModelSerializer):
    class Meta:
        model = Blog
        fields = ['id','title', 'author', 'body'] # author 추가
        read_only_fields = ['id']

In [ ]:
# blog -> models.py

from django.db import models
from accounts.models import CustomUser

# Create your models here.
class Blog(models.Model):
    title = models.CharField(max_length=100)
    author = models.ForeignKey(CustomUser, on_delete=models.CASCADE,null=True) # author 필드 추가 
    body = models.TextField(default="default")
    date = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return self.title

#### 'null=True'를 하는 이유
- 기존 author 없이 작성된 글을 위해
- 없을 경우 기존 글에서 오류 발생

#### 8. accounts 앱에 함수 정의하기

In [ ]:
# accounts -> views.py

from .models import CustomUser
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status
from .serializers import LoginSerializer, SignupSerializer
from django.contrib import auth
from django.contrib.auth.hashers import make_password

# Create your views here.
@api_view(['POST'])
def login(request):
    serializer = LoginSerializer(data=request.data) # LoginSerializer로부터 데이터를 받음
    if serializer.is_valid(): # 유효할 경우
        user = auth.authenticate( # 유저를 반환
            request=request,
            username=serializer.data['username'],
            password=serializer.data['password']
        )
        if user is not None: # 유저가 존재하지 않을 경우
            auth.login(request, user)
            return Response(status=status.HTTP_200_OK) # 정상 작동
        return Response(status=status.HTTP_404_NOT_FOUND) # 해당하는 데이터가 존재 안할 경우
    return Response(status=status.HTTP_400_BAD_REQUEST) # 요청된 값이 유효하지 않을 경우

@api_view(['POST'])
def signup(request):
    serializer = SignupSerializer(data=request.data) # SignupSerializer로부터 데이터를 받음
    if serializer.is_valid():
        new_user = serializer.save(password = make_password(serializer.validated_data['password'])) # make_password 메소드 사용
        auth.login(request, new_user)
        return Response(status=status.HTTP_200_OK)
    return Response(status=status.HTTP_400_BAD_REQUEST)


@api_view(['POST'])
def logout(request):
    auth.logout(request)
    return Response(status=status.HTTP_200_OK)

#### make_password 메소드란?
- 문자열로 받은 비밀번호 값을 해시값으로 변경해주는 함수
    - 해시 값: 암호화한 값으로 고정된 길이의 비트열로 반환되는 값


#### 9. blog의 views.py에 권한 설정하기


- authentication_classes: 권한 설정에 대한 데코레이터
    - SessionAuthentication
    - BasicAuthentication
- permission_classes: 권한이 인증된 유저만 접근이 가능

In [ ]:
# blog -> views.py

from django.shortcuts import render
from .models import Blog
from .serializers import BlogSerializer, BlogPostPutSerializer
from rest_framework.decorators import api_view, authentication_classes,permission_classes
from rest_framework.response import Response
from rest_framework import status
from rest_framework.authentication import SessionAuthentication, BasicAuthentication
from rest_framework.permissions import IsAuthenticated, IsAuthenticatedOrReadOnly
# Create your views here.
'''
전체 블로그를 조회
'''
@api_view(['GET'])
@authentication_classes([SessionAuthentication, BasicAuthentication]) # 권한 설정
def get_all_blogs(request):
    # authentication
    blogs = Blog.objects.all()
    serializer = BlogSerializer(blogs, many=True)
    return Response(serializer.data)

'''
한 블로그 post
'''
@api_view(['POST'])
@authentication_classes([SessionAuthentication, BasicAuthentication])
@permission_classes([IsAuthenticated]) # 권한이 인증된 유저만 작성이 가능
def post_one_blog(request):
    serializer = BlogPostPutSerializer(data=request.data)
    if serializer.is_valid():
        serializer.save(author=request.user)
        return Response(serializer.data, status = status.HTTP_201_CREATED)
    return Response(status=status.HTTP_400_BAD_REQUEST)

'''
한 블로그 조회
'''
@api_view(['GET'])
@authentication_classes([SessionAuthentication, BasicAuthentication])
def get_one_blog(request, pk):
    try:
        blog = Blog.objects.get(pk=pk)
        serializer = BlogSerializer(blog)
        return Response(serializer.data)
    except Blog.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

'''
한 블로그 수정
'''
@api_view(['PUT'])
@authentication_classes([SessionAuthentication, BasicAuthentication])
@permission_classes([IsAuthenticated]) # 권한이 인증된 유저만 수정이 가능
def put_one_blog(request, pk):
    try: 
        blog = Blog.objects.get(pk=pk)
        if blog.author == request.user: # 둘이 동일한 경우만 
            serializer = BlogPostPutSerializer(blog, data=request.data)
            if serializer.is_valid():
                serializer.save()
                return Response(status=status.HTTP_200_OK)
            return Response(status=status.HTTP_400_BAD_REQUEST)
        return Response(status=status.HTTP_401_UNAUTHORIZED)
    except Blog.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

'''
한 블로그 삭제
'''
@api_view(['DELETE'])
@authentication_classes([SessionAuthentication, BasicAuthentication])
@permission_classes([IsAuthenticated]) # 권한이 인증된 유저만 삭제가 가능
def delete_one_blog(request, pk):
    try:
        blog = Blog.objects.get(pk=pk)
        if blog.author == request.user: # 둘이 동일한 경우만
            blog.delete()
            return Response(status=status.HTTP_204_NO_CONTENT)
        return Response(status=status.HTTP_401_UNAUTHORIZED)
    except Blog.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

#### 10. url 정의하기
- tutorial의 url 정의
- accounts 앱 내부의 urls.py 생성 및 정의

In [ ]:
# tutorial -> urls.py

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('blog/', include('blog.urls')),
    path('account/', include('accounts.urls')), # accounts의 urls를 상속
]

In [ ]:
# accounts -> urls.py

from django.urls import path
from .views import *

app_name = 'accounts'

urlpatterns = [
    path('login', login),
    path('signup', signup),
    path('logout', logout)
]

### 서버키고 확인해보기